In [1]:
import os
import sys
os.environ['PATH']+=':'+sys.path[1].split('lib')[0]+'bin'
sys.path.append('/home/diego/Projects')

import textwrap
import openmmtools as mmtools
import yank as yank

# Setting up the experiment

In [5]:
yaml_content = '''
---
# Set the general options of our simulation
options:
  minimize: yes
  verbose: yes
  default_number_of_iterations: 500
  temperature: 300*kelvin
  pressure: 1*atmospheres
  output_dir: output # Does not have to be set, but we do here to show the different outputs

# Configure the specific molecules we will use for our systems
molecules:
  # Define our Host, cucurbit[7]uril, we can call it whatever we want so we just use its name here as the directive
  CB7:
    filepath: input/host.tripos.mol2
    antechamber: # Specify that missing parameters not in GAFF are acquired through ANTECHAMBER first
      charge_method: null # We pre-computed host charges to save time at YANK run, so set this to null to specify no charge computation method
  # Define our ligand molecule, B2
  B2:
    filepath: input/guest.tripos.mol2
    # Get the partial charges for the ligand by generating them from antechamber with the AM1-BCC charge method
    antechamber:
      charge_method: bcc

# Define the solvent for our system, here we use GBSA Implicit Solvent and Particle Mesh Ewald electrostatics treatment for Explicit Sovlent
solvents:
  # We can title this solvent whatever we want. We just call it "GBSA" for easy remembering
  GBSA:
    nonbonded_method: NoCutoff # Main definition of the nonbonded method
    implicit_solvent: OBC2 # Onufriev-Bashford-Case GBSA model, name is related to Implicit solvents in OpenMM
  # Similarly, we can title this solvent whatever we want. We call it "pme" for ease and to be unique from the gbsa entry
  pme:
    nonbonded_method: PME
    # Specifies the distance btween "long range" and "short range" forces for PME
    nonbonded_cutoff: 12*angstroms
    # Specify the distance at which the short range forces are smoothly brought to 0 so there is no discontinuity in the force
    # By default, this is nonbonded_cutoff minus 1 (one) angstrom
    switch_distance: 11*angstroms
    ewald_error_tolerance: 1.0e-4
    # Specify a volume as a radius away the edge of the molecules molecules (CB7 and B2) 
    # this volume will be packed with water, leaving a small cavity for the non-water molecules
    # You should alway minimize if you have this option
    clearance: 12*angstroms
    # Add thise itons if we need to neutralize the system.
    positive_ion: Na+
    negative_ion: Cl-

# Define the systems: What is the ligand, receptor, and solvent we put them in
systems:
  # System names are abitrary
  implicit:
    # These entries point at the previously defined molecules and solvents
    receptor: CB7
    ligand: B2
    solvent: GBSA
    leap:
      parameters: leaprc.gaff2

  explicit:
    receptor: CB7
    ligand: B2
    solvent: pme
    leap:
      parameters: [leaprc.protein.ff14SB, leaprc.gaff, leaprc.water.tip4pew]

  # Call the protocol whatever you would like, here we name it based on the type of calculation we are running
  absolute-binding:
    complex:
      alchemical_path:
        lambda_electrostatics: [1.00, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
        lambda_sterics:        [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.15, 0.10, 0.05, 0.00]
    solvent:
      alchemical_path:
        lambda_electrostatics: [1.00, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
        lambda_sterics:        [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.15, 0.10, 0.05, 0.00]

# Here we combine the system and the protocol to make an expirement
experiments:
  system: !Combinatorial [implicit, explicit]
  protocol: absolute-binding
  restraint:
    # Note that there is no lambda_electrostatics set in the protocol above, that is because we want the guest to always stay near the host.
    type: Harmonic
'''


In [7]:
yaml_builder = yank.experiment.ExperimentBuilder(script=textwrap.dedent(yaml_content))

2018-05-28 16:26:17,967: Setting CUDA platform to use precision model 'mixed'.
2018-05-28 16:26:18,128: ERROR - yank.experiment - System 'absolute-binding' did not validate! Check the schema error below for details
complex: [unknown field]
solvent: [must be of string type]



YamlParseError: System 'absolute-binding' did not validate! Check the schema error below for details
complex: [unknown field]
solvent: [must be of string type]


In [ ]:
# yaml_builder.run_experiments()

# Setting up the system

In [2]:
working_system = TestSystems.CB7B2Implicit()
positions = working_system.positions

In [3]:
working_system.make_nglview()

NGLWidget()